In [ ]:
!nvidia-smi

# Miscellaneous Area

In [ ]:
def modifyLine(line):
  line = line.replace("\'ll", " will")
  line = line.replace("\'re", " are")
  line = line.replace("\'m", " am")
  line = line.replace("\'d", " would")
  line = line.replace("n\'t", " not")
  line = line.replace("\'ve", " have")
  line = line.replace("\'s", " \'s")
  line = line.replace("s\'", " \'s")

  return line

verb_forms_df = pd.read_csv("verbs.txt")

a = verb_forms_df["PresentTense"].str.lower()
b = verb_forms_df["PastTense"].str.lower()
c = verb_forms_df["PastParticiple"].str.lower()
d = verb_forms_df["PresentParticiple"].str.lower()

def ruin_grammer(line):
  for i, word in enumerate(b):
    line = line.replace(" "+word+" "," "+a[i]+" ")
  for i, word in enumerate(c):
    line = line.replace(" "+word+" "," "+a[i]+" ")
  for i, word in enumerate(d):
    line = line.replace(" "+word+" "," "+a[i]+" ")
  line = line.replace(" a ", " ")
  line = line.replace(" an ", " ")
  line = line.replace(" from ", " ")
  line = line.replace(" the ", " ")
  line = line.replace(" on ", " ")
  line = line.replace(" in ", " ")
  line = line.replace(" at ", " ")
  line = line.replace(" of ", " ")
  line = line.replace(" is ", " ")
  line = line.replace(" am ", " ")
  line = line.replace(" to ", " ")
  line = line.replace(" was ", " ")
  line = line.replace(" were ", " ")
  line = line.replace(" it ", " ")
  line = line.replace(" your ", " you ")
      
  return line

#Setup and data

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt

import collections
import random
import numpy as np
import os
import time
import json
from PIL import Image
import pandas as pd
import common_functions as cf

In [ ]:
cf.unzip_data("/content/drive/MyDrive/c415.zip")
X_c4 = []
y_c4 = []
x = True
for i in range(25):
  file1 = open(f'/content/{i}.csv', 'r')
  for line in file1.readlines():
    if line.count("value:") >0:
      line = line.replace(": \"", ": ")
      line = line.replace(".\"\n", ".")
      line = line.replace("?\"\n", "?")
      line = line.replace("!\"\n", "!")
      line = line.replace("\\'s"," is")
      line = line.replace("\\'re"," are")

      if x:
        line = line.split("value: ")[1]
        line = line.replace(" of "," ")
        line = line.replace(" am "," ")
        line = line.replace(" are "," ")
        line = line.replace(" to "," ")
        line = line.replace(" be "," ")
        line = line.replace(" from ","")
        line = modifyLine("<start> "+line+" <end>")
        line = ruin_grammer(line)


        X_c4.append(line)

        x = False
      else:
        y_c4.append("<start> "+line.split("value: ")[1]+" <end>")
        x = True 

len(X_c4), len(y_c4)
X_c4[-10:]
y_c4[-10:]
X_c4_selected = []
y_c4_selected= []
for i, line in enumerate(X_c4):
  if not line.count("\\")>0:
    X_c4_selected.append(line)
    y_c4_selected.append(y_c4[i])

In [ ]:
X_df1 = pd.read_csv("input1.csv")
X_df2 = pd.read_csv("input2.csv")

y_df1 = pd.read_csv("target1.csv")
y_df2 = pd.read_csv("target2.csv")

X_and_y_df3 = pd.read_csv("inputAndTarget3.txt", header=None)

X1 = X_df1["sentence"].to_numpy()
X2 = X_df2[" sentence"].to_numpy()
X3 = X_and_y_df3[1].to_numpy()

y1 = y_df1["sentence"].to_numpy()
y2 = y_df2["sentence"].to_numpy()
y3 = X_and_y_df3[1].to_numpy()

split_size1 = int( len(X1) * .85 )
split_size2 = int( len(X2) * .85 )
split_size3 = int( len(X3) * .80 )

X1_train, X1_val = X1[:split_size1], X1[split_size1:]
X2_train, X2_val = X2[:split_size2], X2[split_size2:]
X3_train, X3_val = X3[:split_size3], X3[split_size3:]

y1_train, y1_val = y1[:split_size1], y1[split_size1:]
y2_train, y2_val = y2[:split_size2], y2[split_size2:]
y3_train, y3_val = y3[:split_size3], y3[split_size3:]

X1_se = []
X2_se = []
X3_se = []
y1_se = []
y2_se = [] 
y3_se = []

for line in X1:
  X1_se.append("<start> " +line + " <end>")
for line in X2:
  X2_se.append("<start> " +line + " <end>")
for line in X3:
  X3_se.append("<start> " +line + " <end>")  

for line in y1:
  y1_se.append("<start> " +line + " <end>")
for line in y2:
  y2_se.append("<start> " +line + " <end>")
for line in y3:
  y3_se.append("<start> " +line + " <end>")    

train_sentences = np.concatenate((X1_se, X3_se, X_c4_selected[:300000] )) 
train_targets = np.concatenate((y1_se, y3_se, y_c4_selected[:300000]))

val_sentences = np.concatenate((X1_val, X3_val))
val_targets = np.concatenate((y1_val, y3_val))

In [ ]:
def standardize(inputs):
  inputs = tf.strings.lower(inputs)
  inputs = tf.strings.regex_replace(inputs,"\'ll", " will")
  inputs = tf.strings.regex_replace(inputs,"\'re", " are")
  inputs = tf.strings.regex_replace(inputs,"\'m", " am")
  inputs = tf.strings.regex_replace(inputs,"\'d", " would")
  inputs = tf.strings.regex_replace(inputs,"n\'t", " not")
  inputs = tf.strings.regex_replace(inputs,"\'ve", " have")
  inputs = tf.strings.regex_replace(inputs,"\'s", " \'s")
  inputs = tf.strings.regex_replace(inputs,"s\'", " \'s")  
  return tf.strings.regex_replace(inputs,
                                  r"!\"#$%&\(\)\*\+.,-/:;=?@\[\\\]^_`{|}~", "")

max_length = 128

vocabulary_size = 7500
tokenizer = tf.keras.layers.TextVectorization(
    max_tokens=vocabulary_size,
    standardize=standardize,
    output_sequence_length=max_length)

s = []
with open('/content/for_vocab.txt') as f:
  for line in f.readlines():
    line = line.replace("1","")
    line = line.replace("2","")
    line = line.replace("3","")
    line = line.replace("4","")
    line = line.replace("5","")
    line = line.replace("6","")
    line = line.replace("7","")
    line = line.replace("8","")
    line = line.replace("9","")
    line = line.replace("0","")
 
    s.append(line.replace(",", " "))
train_sentences
to_learn_vocabulary = np.concatenate((s, train_targets[:10000]))
tokenizer.get_vocabulary()[:1000]
tokenizer.adapt(to_learn_vocabulary)

word_to_index = tf.keras.layers.StringLookup(
    mask_token="",
    vocabulary=tokenizer.get_vocabulary())
index_to_word = tf.keras.layers.StringLookup(
    mask_token="",
    vocabulary=tokenizer.get_vocabulary(),
    invert=True)

In [ ]:
MAX_TOKENS = 128
BUFFER_SIZE = 10000
BATCH_SIZE= 128

def filter_max_tokens(inp, tar):
  num_tokens = tf.maximum(tf.shape(inp)[1],tf.shape(tar)[1])
  return num_tokens <=MAX_TOKENS

def tokenize_pairs(inp, tar): 
  inp = tokenizer(inp)
  tar = tokenizer(tar)
  return inp, tar

def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
      .filter(filter_max_tokens)
      .prefetch(tf.data.AUTOTUNE))

train_ds = tf.data.Dataset.from_tensor_slices((train_sentences, train_targets))
val_ds = tf.data.Dataset.from_tensor_slices((val_sentences, val_targets))

train_ds = make_batches(train_ds)
val_ds = make_batches(val_ds)

# Model

In [ ]:
BATCH_SIZE = 128
BUFFER_SIZE = 1000
embedding_dim = 512
vocab_size = tokenizer.vocabulary_size()
units = 256
num_steps = len(train_sentences) // BATCH_SIZE
features_shape = 8
attention_features_shape = 128

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, MAX_TOKENS, embedding_dim)

    # hidden shape == (batch_size, hidden_size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # attention_hidden_layer shape == (batch_size, MAX_TOKENS, units)
    attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                         self.W2(hidden_with_time_axis)))

    # score shape == (batch_size, MAX_TOKENS, 1)
    # This gives you an unnormalized score for each image feature.
    score = self.V(attention_hidden_layer)

    # attention_weights shape == (batch_size, MAX_TOKENS, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
class Encoder(tf.keras.Model):

    def __init__(self, vocab_size, embedding_dim,):
        super(Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    def call(self, x):
        x = self.embedding(x)
        return x

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru1 = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.gru2 = tf.keras.layers.GRU(int(self.units),
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')    
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(self.units)
    self.fc3 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru1(x)
    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)
    x = self.fc2(x)
    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc3(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [ ]:
encoder = Encoder(vocab_size, embedding_dim)
decoder = Decoder(embedding_dim, units, vocab_size)

# Training

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_path = "content/drive/MyDrive/rnn_checkpoints2/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  # restoring the latest checkpoint in checkpoint_path
  ckpt.restore(ckpt_manager.latest_checkpoint)

loss_plot = []

In [ ]:
@tf.function
def train_step(inp, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from image to image
  hidden = decoder.reset_state(batch_size=target.shape[0])

  dec_input = tf.expand_dims([word_to_index('<start>')] * target.shape[0], 1)

  with tf.GradientTape() as tape:
      features = encoder(inp)

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(target[:, i], 1)

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss

# Actual training

In [ ]:
EPOCHS = 6

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (inp, target)) in enumerate(train_ds):
        batch_loss, t_loss = train_step(inp, target)
        total_loss += t_loss

        if batch % 100 == 0:
            average_batch_loss = batch_loss.numpy()/int(target.shape[1])
            print(f'Epoch {epoch+1} Batch {batch} Loss {average_batch_loss:.4f}')
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 2 == 0:
      ckpt_manager.save()

    print(f'Epoch {epoch+1} Loss {total_loss/num_steps:.6f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec\n')

# Getting output

In [ ]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    features = encoder(tokenizer(sentence))

    dec_input = tf.expand_dims([word_to_index('<start>')], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input,
                                                         features,
                                                         hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        predicted_word = tf.compat.as_text(index_to_word(predicted_id).numpy())
        result.append(predicted_word)

        if predicted_word == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result

In [ ]:
print(evaluate(["I name mayank"]))

(['name', '[UNK]', '<end>'], array([[0.41988662, 0.22259225, 0.04262269, ..., 0.00251919, 0.00251919,
        0.00251919],
       [0.06450473, 0.06692412, 0.02803279, ..., 0.00672431, 0.00672431,
        0.00672431],
       [0.03329823, 0.05236207, 0.05605925, ..., 0.00686624, 0.00686624,
        0.00686624],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]]))


# Saving weights

In [ ]:
encoder.save_weights("/content/drive/MyDrive/enc_weights2")
decoder.save_weights("/content/drive/MyDrive/dec_weights2")

In [ ]:
encoder.load_weights("/content/drive/MyDrive/enc_weights2")
decoder.load_weights("/content/drive/MyDrive/dec_weights2")